In [ ]:
from typing import TypedDict, List

from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq

from dotenv import load_dotenv
load_dotenv()


# ----------------------------
# State Definition
# ----------------------------
class AgentState(TypedDict):
    messages: List[HumanMessage]


# ----------------------------
# Groq LLM Initialization
# ----------------------------
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


# ----------------------------
# Node Logic
# ----------------------------
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(f"\nAI: {response.content}")

    # Append AI response to message history (important for multi-turn)
    state["messages"].append(AIMessage(content=response.content))
    return state


# ----------------------------
# Graph Definition
# ----------------------------
graph = StateGraph(AgentState)

graph.add_node("process", process)
graph.add_edge(START, "process")
graph.add_edge("process", END)

agent = graph.compile()


# ----------------------------
# Chat Loop
# ----------------------------
state = {"messages": []}

user_input = input("Enter your message: ")

while user_input.lower() != "exit":
    state["messages"].append(HumanMessage(content=user_input))
    state = agent.invoke(state)
    user_input = input("\nEnter: ")
